In [3]:
from driftdb.connectors import GithubConnector
from driftdb.version import version
print(version)
from dotenv import load_dotenv
from github import Github
import os
import pandas as pd

load_dotenv("../../.env")

gh_token = os.getenv("GH_TOKEN")
if gh_token is None:
    print("GitHub token not found! Create a .env file a the root with a GH_TOKEN variable.")
    exit(1)
github_client = Github(gh_token, timeout=60)
repo_name = os.getenv("REPO") or "gh_org/repo"
github_connector = GithubConnector(github_client=github_client, github_repository_name=repo_name, assignees=["Sammy"])


0.0.4-a1


In [6]:
table_name = "test/alerts/new_data_alert"

dataMonth1 = pd.DataFrame({"unique_key": ["Alice", "Bob", "Charlie"], "date": ["2022-12","2023-01","2023-01"], "age": [25, 30, 35]})

github_connector.snapshot_table(
    table_dataframe= dataMonth1,
    table_name= table_name,
)

driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: DRIFT
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/4ba55fc4c80508449feb17f1e61deef9419ad9f1


In [7]:
from driftdb.drift_evaluator import DefaultDriftEvaluator
from driftdb.drift_evaluator.interface import DriftEvaluation, NewDataEvaluatorContext

class MyEvaluator(DefaultDriftEvaluator):
    @staticmethod
    def compute_new_data_evaluation(
        new_data_context: NewDataEvaluatorContext,
    ) -> DriftEvaluation:
        print(new_data_context)
        return DriftEvaluation(should_alert=False, message="")

dataMonth1 = pd.DataFrame({"unique_key": ["Alice", "Bob", "Charlie", "Driss"], "date": ["2022-12","2023-01","2023-01", "2023-02"], "age": [25, 30, 35, 24]})

github_connector.snapshot_table(
    table_dataframe= dataMonth1,
    table_name= table_name,
    drift_evaluator=MyEvaluator()
)

driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: NEW DATA


driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/f4820fa883f8252a1c867c52cc00353512411a75
